In [ ]:
import torch
from pathlib import Path
from ibp_neurips19 import models
from ibp_neurips19.attacks import compute_robustness

In [ ]:
# model = 'small'
# dataset = 'CIFAR10'
# epsilon = 0.01
# net = models.fit_to_dataset(models.__dict__[f'{model}_cnn'](), dataset).eval()
# # checkpoint = torch.load(Path.home() / f'results/{dataset}-{model}_cnn-{epsilon}/0.001/checkpoint.pth')
# checkpoint = torch.load(Path.home() / f'results/dm_torch/{dataset.lower()}-{model}-{epsilon}.pth')
# net.load_state_dict(checkpoint['state_dict'])
# # 100 - checkpoint['best_acc1']

In [ ]:
# results = compute_robustness(net, dataset, 'cuda', restarts=1, subset=None, subset_seed=None, attack_kwargs=dict(epsilon=2/255))
# print(results.robustness)
# results.fooling_rate * 100

In [ ]:
import torch
from pathlib import Path
from ibp_neurips19 import models
from argparse import Namespace

def iter_results(main_dir=Path.home() / 'results', exclude=None, with_models=False):
    main_dir = Path(main_dir)
    for file in sorted((main_dir / 'pgd').iterdir()):
        r = Namespace(**torch.load(file))
        if exclude and exclude(r):
            continue
        if r.learning_rate == 0:
            checkpoint_file = (f'{main_dir}/dm_torch/'
                               f'{r.dataset.lower()}-{r.model}-{r.epsilon}.pth')
        else:
            checkpoint_file = (f'{main_dir}/{r.dataset}-{r.model}_cnn-{r.epsilon}'
                               f'/{r.learning_rate}/checkpoint.pth')
        r.checkpoint_file = Path(checkpoint_file)
        checkpoint = torch.load(r.checkpoint_file)
        r.accuracy = checkpoint['accuracy'] if r.learning_rate == 0 else checkpoint['best_acc1'] / 100
        if with_models:
            r.net = models.__dict__[f'{r.model}_cnn']()
            models.fit_to_dataset(r.net, r.dataset).eval()
            r.net.load_state_dict(checkpoint['state_dict'])
        yield r

In [ ]:
exclude = lambda r: r.dataset != 'MNIST' or r.model != 'large' or r.epsilon != 0.1 or r.test_epsilon != 0.1

for r in iter_results(exclude=exclude):
    exp = f'{r.dataset}-{r.model}-{r.epsilon}:{r.test_epsilon:.4e}-{r.learning_rate}'
    print(f"{exp} [{100 * r.accuracy:0.2f}%] = {100 * (1 - r.fooling_rate):.2f}%")